In [1]:
import os
import numpy as np

import numpy as np
import pandas as pd
import os
import collections
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

# Load compressed models from tensorflow_hub
os.environ["TFHUB_MODEL_LOAD_FORMAT"] = "COMPRESSED"

DATA_COLUMN = 'comment_text'
LABEL_COLUMNS = ['toxic','severe_toxic','obscene','threat','insult','identity_hate']

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")
print(os.listdir("./data/"))


Version:  2.3.1
Eager mode:  True
Hub version:  0.9.0
GPU is NOT AVAILABLE
['test_labels.csv', 'test.csv', 'train.csv', 'train_edited.csv', 'train copy.csv']


In [2]:

train_data_path='./data/train copy.csv'
train = pd.read_csv(train_data_path)
test = pd.read_csv('./data/test.csv')
test_labels = pd.read_csv('./data/test_labels.csv')
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [3]:
testing = pd.concat([test, test_labels], axis=1, sort=False)
testing.head()

,id,comment_text,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...,00001cee341fdb12,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...,0000247867823ef7,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap...",00013b17ad220c46,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,00017563c3f7919a,":If you have a look back at the source, the in...",00017563c3f7919a,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
4,00017695ad8997eb,I don't anonymously edit articles at all.,00017695ad8997eb,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [4]:
global_vectors = []
for i in range(16):
    with open(f"vectors{i}", "rb") as f:
        vectors = np.load(f)
        global_vectors.extend(vectors)
len(global_vectors)

159583

In [5]:
global_vectors_test = []
for i in range(15):
    with open(f"vectors_test{i}", "rb") as f:
        vectors = np.load(f)
        global_vectors_test.extend(vectors)
len(global_vectors_test)

150000

In [8]:
def generator():
    for index, row in enumerate(train.iterrows()):
        if index > len(global_vectors):
            break
        if row[1][LABEL_COLUMNS].to_numpy()[0] == None:
            continue
        # print(f"{index} {row[1][LABEL_COLUMNS].to_numpy()}")
        yield global_vectors[index], row[1][LABEL_COLUMNS].to_numpy()
def generator_validation():
    for index, row in enumerate(testing.iterrows()):
        if index > len(global_vectors_test):
            break
        if row[1][LABEL_COLUMNS].to_numpy()[0] == -1:
            continue
        if row[1][LABEL_COLUMNS].to_numpy()[0] == None:
            continue
        # print(f"{index} {row[1][LABEL_COLUMNS].to_numpy()}")
        yield global_vectors_test[index], row[1][LABEL_COLUMNS].to_numpy()

In [9]:
ds = tf.data.Dataset.from_generator(
    generator,
    output_types=(tf.float64, tf.int32),
    output_shapes=((512,), (6,)))
dsvalidation = tf.data.Dataset.from_generator(
    generator_validation,
    output_types=(tf.float64, tf.int32),
    output_shapes=((512,), (6,)))

In [10]:
train_examples_batch, train_labels_batch = next(iter(ds.batch(10)))
print(train_examples_batch)
print(train_labels_batch)

tf.Tensor(
[[ 2.34118924e-02  1.27783231e-02  9.92763671e-05 ...  1.24666803e-02
  -2.50894781e-02 -2.00472008e-02]
 [-2.40830220e-02 -6.94480240e-02  2.24038679e-02 ... -5.46172224e-02
   7.05735534e-02  1.35512464e-02]
 [-2.29524039e-02 -7.94341862e-02  2.46322248e-02 ...  4.14464436e-02
  -7.11882580e-03  5.08726686e-02]
 ...
 [ 3.04732211e-02 -5.81019605e-03 -3.95635217e-02 ... -2.00772900e-02
   2.63790283e-02 -5.80073968e-02]
 [ 1.12369405e-02 -1.94474086e-02 -3.51287723e-02 ... -1.22005015e-03
   4.52916883e-02  7.36773433e-03]
 [-5.36514185e-02  1.48151098e-02  1.82384048e-02 ...  2.22440884e-02
   6.22661486e-02 -5.87005652e-02]], shape=(10, 512), dtype=float64)
tf.Tensor(
[[0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [1 1 1 0 1 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]], shape=(10, 6), dtype=int32)


In [11]:
train_examples_batch, train_labels_batch = next(iter(dsvalidation.batch(10)))
print(train_examples_batch)
print(train_labels_batch)

tf.Tensor(
[[ 0.06062536 -0.04105536 -0.02951291 ... -0.04221195  0.03745399
   0.06499528]
 [ 0.04350374 -0.04548232  0.02512921 ... -0.03704115  0.05740863
   0.03638064]
 [-0.01684072 -0.02395513 -0.00340247 ...  0.04463637  0.00991933
  -0.0773017 ]
 ...
 [-0.02043399 -0.05755927  0.02362465 ... -0.02574916  0.05404703
  -0.06132825]
 [-0.05956182  0.04916395 -0.00378254 ... -0.06009409  0.03066447
  -0.06422771]
 [-0.03162903 -0.07027443  0.0076107  ... -0.02510528  0.03122343
  -0.05803937]], shape=(10, 512), dtype=float64)
tf.Tensor(
[[0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [1 0 0 0 0 0]
 [0 0 0 0 0 0]], shape=(10, 6), dtype=int32)


In [12]:
model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(512,)))
model.add(tf.keras.layers.Dense(512, activation='relu'))
model.add(tf.keras.layers.Dropout(0.38925))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dropout(0.38925))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dropout(0.38925))
model.add(tf.keras.layers.Dense(len(LABEL_COLUMNS), activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               262656    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               65664     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 3

In [13]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [20]:
history = model.fit(
    ds.batch(512),
    epochs=10,
    validation_data=dsvalidation.batch(512),
    verbose=1)

Epoch 1/10
312/312 [==============================] - 532s 2s/step - loss: 0.3595 - accuracy: 0.9090 - val_loss: 0.3711 - val_accuracy: 0.9843
Epoch 2/10
312/312 [==============================] - 512s 2s/step - loss: 0.3591 - accuracy: 0.9122 - val_loss: 0.3706 - val_accuracy: 0.9967
Epoch 3/10
312/312 [==============================] - 507s 2s/step - loss: 0.3588 - accuracy: 0.9176 - val_loss: 0.3704 - val_accuracy: 0.9976
Epoch 4/10
312/312 [==============================] - 470s 2s/step - loss: 0.3586 - accuracy: 0.9053 - val_loss: 0.3705 - val_accuracy: 0.9975
Epoch 5/10
312/312 [==============================] - 471s 2s/step - loss: 0.3583 - accuracy: 0.9124 - val_loss: 0.3700 - val_accuracy: 0.9968
Epoch 6/10
312/312 [==============================] - 526s 2s/step - loss: 0.3580 - accuracy: 0.8988 - val_loss: 0.3702 - val_accuracy: 0.9886
Epoch 7/10
312/312 [==============================] - 791s 3s/step - loss: 0.3578 - accuracy: 0.8896 - val_loss: 0.3704 - val_accuracy: 0.9750

In [21]:
results = model.evaluate(dsvalidation.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

KeyboardInterrupt: 

In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               262656    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               65664     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 3

In [26]:
model.save('export')

INFO:tensorflow:Assets written to: export/assets


In [27]:
import tensorflowjs as tfjs
tfjs.converters.save_keras_model(model, './jsmodel')